In [1]:
from finance.mongo.extraction import query_mongodb
from finance.params import *
import numpy as np
import pandas as pd


In [2]:
exchange_ls = EXCHANGE_LS
table_name = 'key_ratio'
kwargs = {'calendarYear': { "$gt": 2021 }}

In [3]:
client = PY_MONGO_CLIENT
collection = client['finance'][table_name]

In [4]:
result = query_mongodb(exchange_ls=[], table=table_name, **kwargs)

exchange {'calendarYear': {'$gt': 2021}} - 339470


In [5]:
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, \
    TimeSeriesResampler

/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/finance/lib/python3.10/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [11]:
df  = pd.DataFrame(result)
df = df.drop(columns='_id').drop_duplicates()

In [68]:
df[df['symbol']=='GOOG']

,symbol,date,calendarYear,period,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,...,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue,key
17761,GOOG,2023-09-30,2023,Q3,2.043108,1.865172,0.355780,48.137379,8.008968,56.146347,...,21.062584,73.395197,54.110283,54.110283,2.776432,21.629156,0.0,62.922978,6.071716,2023Q3GOOG
17762,GOOG,2023-06-30,2023,Q2,2.172052,2.022108,0.333668,46.811967,6.291202,53.103169,...,20.857578,70.366790,53.458730,53.458730,0.911553,20.541096,0.0,58.689508,5.736476,2023Q2GOOG
17763,GOOG,2023-03-31,2023,Q1,2.352587,2.195050,0.376507,46.473412,6.806154,53.279566,...,22.078666,77.190708,56.541069,56.541069,1.950282,19.046871,0.0,61.573181,5.094881,2023Q1GOOG
17764,GOOG,2022-12-31,2022,Q4,2.377994,2.222511,0.315714,47.643857,6.799276,54.443132,...,20.998804,71.437094,48.460693,48.460693,-22.468720,15.047744,0.0,49.478262,4.467607,2022Q4GOOG
17765,GOOG,2022-09-30,2022,Q3,2.517604,2.310356,0.333197,47.123256,9.116118,56.239374,...,22.496059,77.855365,53.598283,53.598283,-1.829679,18.116145,0.0,57.486923,4.935143,2022Q3GOOG
17766,GOOG,2022-06-30,2022,Q2,2.809450,2.641556,0.292336,47.880749,5.919479,53.800228,...,22.440251,114.050783,73.955080,73.955080,-9.350104,20.612120,0.0,59.560751,5.623527,2022Q2GOOG
17767,GOOG,2022-03-31,2022,Q1,2.871005,2.737651,0.337154,47.139139,4.162641,51.301780,...,28.046008,120.356186,73.442873,73.442873,-1.411348,27.111155,0.0,76.160754,7.259164,2022Q1GOOG


In [80]:
moat = pd.DataFrame()

moat['symbol'] = df['symbol']
moat['date'] = df['date']
moat['operatingmargin'] = df['operatingProfitMargin'].apply(lambda x: x/0.15)
moat['fcfmargin'] = df['freeCashFlowOperatingCashFlowRatio'].apply(lambda x: x/0.05)
moat['netmargin'] = df['netProfitMargin'].apply(lambda x: x/0.15)
moat['roa'] = df['returnOnAssets'].apply(lambda x: x/0.06)
moat['roe'] = df['returnOnEquity'].apply(lambda x: x/0.01)

moat

,symbol,date,operatingmargin,fcfmargin,netmargin,roa,roe
0,MYID.V,2023-03-31,-1.382152,20.678828,-1.382152,-3.251723,5.715233
1,MYID.V,2022-12-31,-5.908774,19.795459,-5.906799,-11.567085,24.875167
2,MYID.V,2022-09-30,-7.681478,20.001988,-7.415628,-8.358127,32.085114
3,MYID.V,2022-06-30,-11.697354,20.000000,-8.420702,-7.420715,47.552046
4,MYID.V,2022-03-31,-35.356929,20.049337,-35.204409,-27.012223,95.133131
...,...,...,...,...,...,...,...
339316,CRFTF,2023-03-31,0.000000,20.000000,0.000000,-5.315910,0.690149
339317,CRFTF,2022-12-31,-14.263549,20.000000,-14.263549,-10.642998,1.391001
339318,CRFTF,2022-09-30,0.000000,20.000000,0.000000,-910.640205,86.136393
339319,CRFTF,2022-06-30,-1038.988417,20.000000,19598.025740,192.162239,-556.979181


In [110]:
moat.dropna(inplace=True)

In [111]:
moat[moat['symbol']=='NVDA'][:8]

,symbol,date,operatingmargin,fcfmargin,netmargin,roa,roe
206560,NVDA,2023-10-29,3.832597,19.241680,3.400662,2.844980,17.069883
206561,NVDA,2023-07-30,3.356284,19.089477,3.054219,2.081189,22.501000
206562,NVDA,2023-04-30,1.983686,18.296118,1.893771,0.765857,8.331974
206563,NVDA,2023-01-29,1.384895,15.473544,1.557869,0.572256,6.397901
206564,NVDA,2022-10-30,0.675547,-7.040816,0.764346,0.279918,3.185161
206565,NVDA,2022-07-31,0.496221,13.181102,0.652347,0.251480,2.750409
206566,NVDA,2022-05-01,1.502574,15.829001,1.301480,0.596449,6.147416
206567,NVDA,2022-01-30,2.590606,18.199802,2.619390,1.132686,11.284383


In [78]:
moat = moat.drop_duplicates(subset=['date', 'operatingmargin', 'fcfmargin', 'netmargin', 'roa', 'roe'])

In [24]:
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import RobustScaler

ss = make_column_transformer((RobustScaler(), make_column_selector(dtype_include='float64')))

moat[['operatingmargin', 'fcfmargin', 'netmargin', 'roa', 'roe']] =  ss.fit_transform(moat)

moat.sort_values(['symbol', 'date'], inplace=True)

/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_95536/2738591933.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moat[['operatingmargin', 'fcfmargin', 'netmargin', 'roa', 'roe']] =  ss.fit_transform(moat)
/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_95536/2738591933.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moat.sort_values(['symbol', 'date'], inplace=True)


In [72]:
moat.symbol.nunique()

21131

In [87]:
moat[moat['symbol']=='GOOG'].symbol.unique()

array(['GOOG'], dtype=object)

In [112]:
from tqdm import tqdm

finance_ls = []
symbol_ls = []

for i in tqdm(moat.symbol.unique()):
    sub = moat[moat['symbol']==i]
    if len(sub)>=8:
        sub = sub[:8]
        symbol_ls.append(sub.symbol.unique()[0])
        sub.drop(columns=['symbol', 'date'], inplace=True)
        finance_ls.append(sub.to_numpy())

100%|██████████| 25452/25452 [09:13<00:00, 45.99it/s]


In [113]:
stock_ls = np.array(finance_ls)
stock_ls = TimeSeriesScalerMeanVariance().fit_transform(stock_ls)

In [114]:
stock_ls[0]

array([[ 0.29871167,  0.48373617, -0.29483772,  0.12701595,  0.21653085],
       [-0.92119412,  0.39514011, -0.76385082, -0.65990472, -0.28597586],
       [-1.44518356,  0.48778873,  0.67977938,  0.99532498,  0.74692398],
       [-1.39998598,  0.58975829,  2.41464749,  2.17492391,  2.18814695],
       [ 0.92429585, -2.61402346, -0.53116539, -0.86334004, -1.28484088],
       [ 0.89307255,  0.10850102, -0.49634734, -0.52880797, -0.5398522 ],
       [ 0.82083479,  0.15524979, -0.51468095, -0.73913157, -0.66438071],
       [ 0.8294488 ,  0.39384934, -0.49354464, -0.50608054, -0.37655212]])

In [117]:
dba_km = TimeSeriesKMeans(n_clusters=10,
                          n_init=3,
                            metric="softdtw",
                           metric_params={"gamma": .01},
                          verbose=True,
                          random_state=0)

In [118]:
y_pred = dba_km.fit_predict(stock_ls)

Init 1
1034.267 --> 630.493 --> 592.174 --> 568.131 --> 554.761 --> 548.308 --> 545.102 --> 543.399 --> 542.787 --> 541.067 --> 538.927 --> 537.053 --> 534.446 --> 531.219 --> 529.590 --> 528.249 --> 526.932 --> 526.136 --> 525.105 --> 524.692 --> 524.312 --> 524.271 --> 523.779 --> 523.093 --> 522.865 --> 522.854 --> 522.752 --> 522.586 --> 522.471 --> 522.408 --> 522.246 --> 521.815 --> 521.295 --> 520.968 --> 520.813 --> 520.661 --> 520.538 --> 520.373 --> 520.168 --> 519.959 --> 519.915 --> 519.788 --> 519.148 --> 518.907 --> 518.788 --> 518.623 --> 518.384 --> 518.255 --> 518.114 --> 517.671 --> 
Init 2
1072.147 --> 556.964 --> 535.105 --> 523.774 --> 518.623 --> 515.474 --> 513.990 --> 513.033 --> 511.604 --> 511.134 --> 510.420 --> 510.168 --> 509.843 --> 509.504 --> 509.186 --> 509.078 --> 509.000 --> 508.943 --> 508.815 --> 508.653 --> 508.550 --> 508.543 --> 508.496 --> 508.466 --> 508.404 --> 508.347 --> 507.961 --> 507.817 --> 507.817 --> 507.815 --> 507.812 --> 507.813 -->

In [125]:
cluster_df = pd.DataFrame([symbol_ls,y_pred], index=['symbol', 'cluster']).T

In [145]:
cluster_df[cluster_df['symbol']=='NVDA']

,symbol,cluster
1097,NVDA,4


In [148]:
cluster_df[cluster_df['cluster']==4].head(50)

,symbol,cluster
3,INTA,4
14,CASY,4
15,CRS,4
24,YTRA,4
38,RYAAY,4
48,SYY,4
53,GWRE,4
62,TNX.TO,4
72,0IL6.L,4
79,MLHR,4
